<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/C%C3%B3pia_de_Research_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Modules:
Spark, pyspark, Java.

Bellow it is the old fashion way, risky not Recomended

In [187]:
""" 
# # Installing Java
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
# # Dowload Spark
! wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
# # Decompressing archives
! tar xf spark-3.2.1-bin-hadoop2.7.tgz
# # installing findspark
! pip install -q findspark
# # installing pyspark
! pip install pyspark==3.2.1
 """

' \n# # Installing Java\n! apt-get install openjdk-8-jdk-headless -qq > /dev/null\n# # Dowload Spark\n! wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz\n# # Decompressing archives\n! tar xf spark-3.2.1-bin-hadoop2.7.tgz\n# # installing findspark\n! pip install -q findspark\n# # installing pyspark\n! pip install pyspark==3.2.1\n '

In [188]:
""" 
# Importing os library
import os
# # Java envirorment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# # Spark envirorment variable
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
 """

' \n# Importing os library\nimport os\n# # Java envirorment variable\nos.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"\n# # Spark envirorment variable\nos.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"\n '

# Better and consistent library that makes the same function as the code above



In [189]:
! pip install pyspark py4j
! pip install pyodbc


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Importing modules and pandas library that will be used later.**

In [190]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
import pyodbc
# import findspark

# findspark.init()

In [191]:
# instatianting spark instance
spark = SparkSession.builder.master('local[*]').getOrCreate()

spark

Gooogle Colab Version

In [192]:
""" 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) 
"""

" \nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True) \n"

# Next cells acquire parquet data from remote files, and transpose it to spark format

In [193]:
fille =r'C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/Balance.parquet'

balance = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

Creating Tables in order to manipulate data in SQL format

In [194]:
# Balance table
balance.createOrReplaceTempView("Balance")

# Note 1: 
'DetToEquity' must be in percentage, try to get it done here or otherwise in Power Bi

---

# Note 2: 
Find a way to calculate Beta values from 
historical data get it from other sources, but try to calculate it localy (Talk with Felipe, about the period of time that will be used as 'ytd' or montlhy). 

# Fixed:
Beta values could be acquired trough yahoo finance

Listing brazilian companies that will be researched and getting hystorical financial data
filters that our Analyst indicates: Raking companies by: EBIT/EV > 0, ROE > 0, **Daily equity < 1.000.000.000**, revenueGrowth > 0 AND Will be done apart :(5-year-history) and all margins must be postive

Alternative operations, while dealing with a little amount of data, and it can be joined here

In [195]:
Balance = spark.sql("""
SELECT  Ticker,
        CAST(Data_de_Reporte as string),
        Moeda,
        CAST(Receita_a_Pagar as string),
        CAST(Receitas_a_Receber as string),
        CAST(Bens_de_Capital as string)
FROM Balance
WHERE Ticker NOT LIKE '%34.SA'
AND Moeda = 'BRL'
ORDER BY Ticker DESC;
""")

In [196]:
Balance.show()

+---------+-------------------+-----+---------------+------------------+---------------+
|   Ticker|    Data_de_Reporte|Moeda|Receita_a_Pagar|Receitas_a_Receber|Bens_de_Capital|
+---------+-------------------+-----+---------------+------------------+---------------+
| ZAMP3.SA|2021-12-30 21:00:00|  BRL|      2.47917E8|         1.39267E8|     1.461068E9|
| ZAMP3.SA|2020-12-30 21:00:00|  BRL|      2.17616E8|         1.05011E8|     1.461068E9|
| ZAMP3.SA|2019-12-30 21:00:00|  BRL|      2.86432E8|          6.0361E7|      9.50768E8|
| YDUQ3.SA|2019-12-30 21:00:00|  BRL|      1.26651E8|         7.59622E8|     1.139887E9|
| YDUQ3.SA|2021-12-30 21:00:00|  BRL|      1.83529E8|         9.57746E8|     1.139887E9|
| YDUQ3.SA|2020-12-30 21:00:00|  BRL|      2.51229E8|         8.90151E8|     1.139887E9|
|XPBR31.SA|2019-12-30 21:00:00|  BRL|      2.66813E8|         4.58776E8|        23000.0|
|XPBR31.SA|2022-12-30 21:00:00|  BRL|      6.17394E8|         4.87331E8|        24000.0|
|XPBR31.SA|2021-12-30

ev = número total de papeis x cotação + divida liquida
ev = market value +  liquid debt 

PSR = PREÇO DA AÇÃO / RECEITA LIQUIDA POR AÇÃO 

Pandas setting DataFrame to pandas format:

In [197]:
df_balance = Balance.toPandas()

In [198]:
df_balance

,Ticker,Data_de_Reporte,Moeda,Receita_a_Pagar,Receitas_a_Receber,Bens_de_Capital
0,ZAMP3.SA,2019-12-30 21:00:00,BRL,2.86432E8,6.0361E7,9.50768E8
1,ZAMP3.SA,2020-12-30 21:00:00,BRL,2.17616E8,1.05011E8,1.461068E9
2,ZAMP3.SA,2021-12-30 21:00:00,BRL,2.47917E8,1.39267E8,1.461068E9
3,YDUQ3.SA,2019-12-30 21:00:00,BRL,1.26651E8,7.59622E8,1.139887E9
4,YDUQ3.SA,2020-12-30 21:00:00,BRL,2.51229E8,8.90151E8,1.139887E9
...,...,...,...,...,...,...
749,AERI3.SA,2021-12-30 21:00:00,BRL,4.45286E8,1.26877E8,8.15102E8
750,AERI3.SA,2022-12-30 21:00:00,BRL,3.36048E8,0.0,8.15102E8
751,AALR3.SA,2019-12-30 21:00:00,BRL,5.667E7,8.8789E7,6.12412E8
752,AALR3.SA,2020-12-30 21:00:00,BRL,9.4887E7,7.9679E7,6.12412E8


Excel format tables

In [162]:
""" 
df_stat.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Excel/Stat.xlsx",
                 sheet_name="stat")

df_calc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Calcl.xlsx",
                 sheet_name="calc")

df_rsc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/RSCH.xlsx",
                sheet_name="research")

df_balance.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Balance.xlsx",
                sheet_name="balance")
"""

' \ndf_stat.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Excel/Stat.xlsx",\n                 sheet_name="stat")\n\ndf_calc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Calcl.xlsx",\n                 sheet_name="calc")\n\ndf_rsc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/RSCH.xlsx",\n                sheet_name="research")\n\ndf_balance.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Balance.xlsx",\n                sheet_name="balance")\n'

Parquet format table

In [163]:
""" 
df_balance.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Balance.parquet")

df_stat.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Calcl.parquet")

df_rsc.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/RSCH.parquet")

df_calc.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Calcl.parquet")


df_balance.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Balance.parquet")

df_calendario.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Calendario.parquet")

"""

In [164]:
df_balance['Bens_de_Capital'] = df_balance['Bens_de_Capital'].astype("string")
df_balance['Receitas_a_Receber'] = df_balance['Receitas_a_Receber'].astype("string")
df_balance['Receita_a_Pagar'] = df_balance['Receita_a_Pagar'].astype("string")

print(df_balance.dtypes)

Ticker                object
Data_de_Reporte       object
Moeda                 object
Receita_a_Pagar       string
Receitas_a_Receber    string
Bens_de_Capital       string
dtype: object


Local Connection to SQL Server

In [165]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-7VLQ9AP\SQLEXPRESS;'
                      'Database=FeA;'
                      'Trusted_Connection=yes;'
                      )

cursor = conn.cursor()

Balance Table insert

In [ ]:
for i, ticker in enumerate(df_balance['Ticker']):
    data = df_balance.loc[i,'Data_de_Reporte']

    moeda = df_balance.loc[i,'Moeda']

    receita = df_balance.loc[i,'Receita_a_Pagar']

    receitas = df_balance.loc[i,'Receitas_a_Receber']

    bens = df_balance.loc[i,'Bens_de_Capital']

    dados = '\'' + ticker + '\'' + ',\'' + data  + '\'' + ',\'' + moeda + '\'' + ',\'' + receita + '\'' + ',\'' + receitas + '\'' + ',\'' + bens + '\''  ')'

    script = '''INSERT INTO Balances ([Ticker],[Data_do_Reporte],[Moeda],[Receita_a_pagar],[Receitas_a_receber],[Bens_de_Capital]) VALUES ('''

    query = script + dados

    #print(query)

    cursor.execute(query)

    cursor.commit()

Cloud Version

In [ ]:
""" 
server_name = "jdbc:sqlserver://DESKTOP-7VLQ9AP\SQLEXPRESS"
database_name = "FeA"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "Balance"
username = "SQL_ADM"
password = "and010500" # Please specify password here

try:
  df_balance.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", table_name) \
    .option("user", username) \
    .option("password", password) \
    .save()

except ValueError as error :
    print("Connector write failed", error)
 """

' \nserver_name = "jdbc:sqlserver://DESKTOP-7VLQ9AP\\SQLEXPRESS"\ndatabase_name = "FeA"\nurl = server_name + ";" + "databaseName=" + database_name + ";"\n\ntable_name = "Balance"\nusername = "SQL_ADM"\npassword = "and010500" # Please specify password here\n\ntry:\n  df_balance.write     .format("com.microsoft.sqlserver.jdbc.spark")     .mode("overwrite")     .option("url", url)     .option("dbtable", table_name)     .option("user", username)     .option("password", password)     .save()\n\nexcept ValueError as error :\n    print("Connector write failed", error)\n '